In [5]:
import configparser

config = configparser.ConfigParser()

# URLs section
config['URLs'] = {
    'login_url': 'https://www.sunvalley.co.kr/member/login'
}

# Reservation URLs section
config['ReservationURLs'] = {
    'reservation_url_설악썬밸리': 'https://www.sunvalley.co.kr/reservation/golf?sel=J21',
    'reservation_url_썬밸리CC': 'https://www.sunvalley.co.kr/reservation/golf?sel=J23',
    'reservation_url_동원썬밸리': 'https://www.sunvalley.co.kr/reservation/golf?sel=J24',
    'reservation_url_여주썬밸리': 'https://www.sunvalley.co.kr/reservation/golf?sel=J25'
}

# Login section
config['Login'] = {
    'id': '51041200',
    'pw': 'yb202601*',
    'session_id': ''
}

# Save the configuration to a file
with open('config.ini', 'w') as config_file:
    config.write(config_file)


In [13]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import sunvalley_login_info

# 통합예약 url
reservation_url_설악썬밸리 = 'https://www.sunvalley.co.kr/reservation/golf?sel=J21'
reservation_url_썬밸리CC = 'https://www.sunvalley.co.kr/reservation/golf?sel=J23'
reservation_url_동원썬밸리 = 'https://www.sunvalley.co.kr/reservation/golf?sel=J24'
reservation_url_여주썬밸리 = 'https://www.sunvalley.co.kr/reservation/golf?sel=J25'

desired_date = input("날짜 (ex:20230610): ")
reservation_course = input('1. 설악썬밸리 2. 썬밸리CC 3. 동원썬밸리 4. 여주썬밸리')
if reservation_course == '1':
    reservation_url = reservation_url_설악썬밸리
elif reservation_course == '2':
    reservation_url = reservation_url_썬밸리CC
elif reservation_course == '3':
    reservation_url = reservation_url_동원썬밸리
elif reservation_course == '4':
    reservation_url = reservation_url_여주썬밸리
else:
    print('잘못된 입력입니다')

# WebDriver 초기화
driver = webdriver.Chrome('chromedriver')  # Chrome 드라이버 경로 설정

# 로그인 url
login_url = 'https://www.sunvalley.co.kr/member/login'

# 로그인 페이지 열기
driver.get(login_url)

# 로그인
id_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
    (By.CSS_SELECTOR, "#usrId" )
))
pw_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
    (By.CSS_SELECTOR, "#usrPwd" )
))
id_input.send_keys(sunvalley_login_info.id)
pw_input.send_keys(sunvalley_login_info.pw)

login_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "#fnLogin"))
)
login_button.click()

WebDriverWait(driver, 10).until(EC.alert_is_present())
# 예약 확인 대화상자 처리
alert = driver.switch_to.alert
alert.accept()  # 확인 버튼 클릭


# 예약 페이지 접속
driver.get(reservation_url)

# 원하는 날짜 선택
date_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
    (By.CSS_SELECTOR, "#A" + desired_date + " > a" )
))
date_input.click()

    
# 원하는 시간 선택
time_select = Select(driver.find_element_by_id('bookgTime'))  # 시간 선택 드롭다운의 ID로 변경
time_select.select_by_visible_text('10:00 AM')  # 예약하려는 시간 선택

# 필요한 기타 작업 수행

# 예약 버튼 클릭
reserve_button = driver.find_element_by_id('reserveButton')  # 예약 버튼의 ID로 변경
reserve_button.click()

# 예약 확인 대화상자 처리
alert = driver.switch_to.alert
alert.accept()  # 확인 버튼 클릭

# WebDriver 종료
driver.quit()


날짜 (ex:20230610):  20230616
1. 설악썬밸리 2. 썬밸리CC 3. 동원썬밸리 4. 여주썬밸리 1


AttributeError: 'WebDriver' object has no attribute 'find_element_by_id'

In [18]:
import os
import logging
import configparser
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


# Load configuration using configparser
config = configparser.ConfigParser()
config.read('config.ini')

# Set up logging
logging.basicConfig(filename='script.log', level=logging.INFO)

def login(driver):
    # 로그인 페이지 열기
    driver.get(config['URLs']['login_url'])

    # 로그인
    id_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, "#usrId")
    ))
    pw_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, "#usrPwd")
    ))
    id_input.send_keys(config['Login']['id'])
    pw_input.send_keys(config['Login']['pw'])

    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#fnLogin"))
    )
    login_button.click()

    WebDriverWait(driver, 10).until(EC.alert_is_present())
    # 예약 확인 대화상자 처리
    alert = driver.switch_to.alert
    alert.accept()  # 확인 버튼 클릭
    
def save_session_id(driver):
    # Get the session ID and store it for reuse
    session_id = driver.session_id
    config['Login']['session_id'] = session_id
    with open('config.ini', 'w') as f:
        config.write(f)


def navigate_to_reservation_page(driver, reservation_url):
    # 예약 페이지 접속
    driver.get(reservation_url)

def select_desired_date(driver, desired_date):
    # 원하는 날짜 선택
    date_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, "#A" + desired_date + " > a")
    ))
    date_input.click()

def select_desired_time(driver, desired_time):
    # 원하는 시간 선택
    time_select = Select(driver.find_element_by_id('bookgTime'))  # 시간 선택 드롭다운의 ID로 변경
    time_select.select_by_visible_text(desired_time)  # 예약하려는 시간 선택

def perform_additional_tasks(driver):
    # 필요한 기타 작업 수행
    pass

def click_reserve_button(driver):
    # 예약 버튼 클릭
    reserve_button = driver.find_element_by_id('reserveButton')  # 예약 버튼의 ID로 변경
    reserve_button.click()

def handle_reservation_confirmation(driver):
    # 예약 확인 대화상자 처리
    alert = driver.switch_to.alert
    alert.accept()  # 확인 버튼 클릭

def main():
    desired_date = input("날짜 (ex:20230610): ")
    reservation_course_mappings = {
    '1': '설악썬밸리',
    '2': '썬밸리CC',
    '3': '동원썬밸리',
    '4': '여주썬밸리',
    }

    # Get the reservation course choice from the user
    reservation_course = input('1. 설악썬밸리 2. 썬밸리CC 3. 동원썬밸리 4. 여주썬밸리: ')

    if reservation_course not in reservation_course_mappings:
        print('잘못된 입력입니다')
        return

    # Get the reservation course name using the dictionary mapping
    reservation_course_name = reservation_course_mappings[reservation_course]

    # Create the reservation URL key based on the reservation course name
    reservation_url_key = f"reservation_url_{reservation_course_name}"

    if reservation_url_key not in config['ReservationURLs']:
        print('잘못된 입력입니다')
        return

    # Get the reservation URL using the reservation URL key
    reservation_url = config['ReservationURLs'][reservation_url_key]


    reservation_url = config['ReservationURLs'][reservation_url_key]

    # Create a new WebDriver instance or attach to an existing one
    try:
        # Attempt to attach to an existing browser session
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.session_id = config['Login'].get('sessionId', '')
    except:
        # Create a new browser session if no existing session is found
        driver = webdriver.Chrome(ChromeDriverManager().install())
        

    try:
        # 로그인
        login(driver)

        # 예약 페이지로 이동
        navigate_to_reservation_page(driver, reservation_url)

        # 원하는 날짜 선택
        select_desired_date(driver, desired_date)

        # 원하는 시간 선택
        desired_time = input('예약하려는 시간 (ex:10:00 AM): ')
        select_desired_time(driver, desired_time)

        # 추가 작업 수행
        perform_additional_tasks(driver)

        # 예약 버튼 클릭
        click_reserve_button(driver)

        # 예약 확인 대화상자 처리
        handle_reservation_confirmation(driver)

        # 세션 ID 저장
        save_session_id(driver)
    except Exception as e:
        logging.exception("An error occurred during the reservation process: %s", str(e))
    finally:
        # WebDriver 종료
        driver.quit()

if __name__ == '__main__':
    main()


날짜 (ex:20230610):  20230617
1. 설악썬밸리 2. 썬밸리CC 3. 동원썬밸리 4. 여주썬밸리:  1


[WDM] - Downloading: 100%|█████████████████| 7.40M/7.40M [00:00<00:00, 57.2MB/s]
/var/folders/3m/knb96f9s4xg2t2gcrl96r6jh0000gn/T/ipykernel_70519/2022766982.py:117: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [16]:
!pip install webdriver_manager